# [Artificial Neural Networks - Assignment](https://docs.google.com/viewer?a=v&pid=sites&srcid=ZGVmYXVsdGRvbWFpbnxhcnRpZmljaWFsbmV1cmFsbmV0d29ya3Nhbm58Z3g6MzZmMzBjY2ZmN2EyMmMyYQ)


In [0]:
 import numpy as np
 from typeguard import typechecked
 from typing import List, Callable, Union

Feed forward network layer.

In [0]:
class Layer:
    @typechecked
    def __init__(self, w: np.ndarray, b: np.ndarray, 
                 activation: Union[Callable, None] = None):
        assert w.ndim == 2, "Weights must be 2D matrix"
        assert b.ndim == 1, "Biases must be 1D vector"
        assert w.shape[0] == b.shape[0], "Weights and biases must"\
            " conform to same number of neurons"

        self.weights = w
        self.biases = b

        self.activation = activation
    
    def get_size(self):
        output_size, input_size = self.weights.shape
        return input_size, output_size

    @typechecked
    def forward(self, inputs: np.ndarray) -> np.ndarray:
        n_samples, n_features = inputs.shape

        assert n_features == self.weights.shape[-1], "Inputs must be a"\
            f" n x {n_features} matrix"
        n_neurons = self.weights.shape[0]

        ones = np.ones((n_samples, 1))
        
        inputs = np.hstack([ones, inputs])
        b = np.expand_dims(self.biases, -1)
        wb = np.hstack([b, self.weights])

        output = inputs @ wb.T
        if self.activation is not None:
            return self.activation(output)
        else:
            return output

    @typechecked
    def __call__(self, inputs: np.ndarray) -> np.ndarray:
        return self.forward(inputs)

Set random seed for determininstic results.

In [0]:
np.random.seed(7)

In [0]:
n_samples = 10
n_features = 5
n_neurons = 3

X = np.random.normal(size=(n_samples, n_features))
w = np.random.random(size=(n_neurons, n_features))
b = np.random.random(size=(n_neurons, ))

In [0]:
layer1 = Layer(w, b)
layer1_output = layer1(X)
assert layer1.get_size() == (n_features, n_neurons)

In [0]:
class Net:
    @typechecked
    def __init__(self, layers: List[Layer]):
        assert len(layers) > 0, "Number of layers should be atleast one"
        Net.check_shapes(layers)
        self.layers = layers
    
    @staticmethod
    def check_shapes(layers: List[Layer]):
        prev_layer = layers[0]
        for layer in layers[1:]:
            _, prev_output = prev_layer.get_size()
            this_input, _ = layer.get_size()
            
            assert prev_output == this_input, "Layer input size must exactly"\
                " match output shape of the preceeding layer"
            
            prev_layer = layer
    
    @typechecked
    def forward(self, inputs: np.ndarray):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs
    
    @typechecked
    def __call__(self, inputs: np.ndarray):
        return self.forward(inputs)

In [0]:
n_samples = 10
n_features = 5
n_neurons_layer1 = 3
n_neurons_layer2 = 5
n_neurons_layer3 = 1

X = np.random.normal(size=(n_samples, n_features))

w1 = np.random.random(size=(n_neurons_layer1, n_features))
b1 = np.random.random(size=(n_neurons_layer1, ))

w2 = np.random.random(size=(n_neurons_layer2, n_neurons_layer1))
b2 = np.random.random(size=(n_neurons_layer2, ))

w3 = np.random.random(size=(n_neurons_layer3, n_neurons_layer2))
b3 = np.random.random(size=(n_neurons_layer3, ))

In [0]:
identity = lambda x: x
relu = lambda x: np.maximum(x, 0)

In [0]:
layer1 = Layer(w1, b1, relu)
layer2 = Layer(w2, b2, relu)
layer3 = Layer(w3, b3, identity)

In [89]:
y = layer3(layer2(layer1(X)))
y

array([[4.9671351 ],
       [3.08797328],
       [2.0310967 ],
       [1.72664332],
       [8.48181661],
       [1.59383825],
       [1.47846404],
       [5.1861727 ],
       [3.24786588],
       [4.86342837]])

In [91]:
net = Net([layer1, layer2, layer3])
assert all(net(X) == y)
net(X)

array([[4.9671351 ],
       [3.08797328],
       [2.0310967 ],
       [1.72664332],
       [8.48181661],
       [1.59383825],
       [1.47846404],
       [5.1861727 ],
       [3.24786588],
       [4.86342837]])

Step activation function is defined as:
$ f(x) = \left\{ \begin{array}{ll}
      0 & for & x\leq 0 \\
      1 & for & x\geq 0 \\
\end{array}
\right. $

In [0]:
step = lambda x: (x > 0).astype(x.dtype)

In [0]:
x1 = 0
x2 = 1

w1 = np.array([[-1, -1], [1, 1]])
b1 = np.array([0.5, -0.5])

w2 = np.array([[1, 1]])
b2 = np.array([-0.5])

In [0]:
hidden_layer = Layer(w1, b1, step)
output_layer = Layer(w2, b2, step)
net = Net([hidden_layer, output_layer])

In [0]:
X = np.array([[x1, x2]])

In [96]:
y = net(X)
y

array([[1.]])